In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, BatchNormalization, MaxPool2D, Dropout, Flatten, Dense
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing import image

In [2]:
# Directorios de entrenamiento y prueba
train_dir = 'C:/Users/Usuario/Desktop/chest_xray/train'
test_dir = 'C:/Users/Usuario/Desktop/chest_xray/test'
val_dir = 'C:/Users/Usuario/Desktop/chest_xray/val'

In [3]:
# Configurar generadores de imágenes
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    color_mode='grayscale'
)

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    color_mode='grayscale'
)

val_datagen = ImageDataGenerator(rescale=1./255)
val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    color_mode='grayscale'
)

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
Found 16 images belonging to 2 classes.


In [4]:
# Definir el modelo
model = Sequential()
model.add(Conv2D(32 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu' , input_shape = (150,150,1)))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))
model.add(Conv2D(64 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))
model.add(Dropout(0.1))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))
model.add(Conv2D(64 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))
model.add(Conv2D(128 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))
model.add(Conv2D(256 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))
model.add(Flatten())
model.add(Dense(units = 128 , activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(units = 1 , activation = 'sigmoid'))
model.compile(optimizer = RMSprop() , loss = 'binary_crossentropy' , metrics = ['accuracy'])
model.summary()
model.save('modelo_neumonia.final')



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 150, 150, 32)      320       
                                                                 
 batch_normalization (Batch  (None, 150, 150, 32)      128       
 Normalization)                                                  
                                                                 
 max_pooling2d (MaxPooling2  (None, 75, 75, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 75, 75, 64)        18496     
                                                                 
 dropout (Dropout)           (None, 75, 75, 64)        0         
                                                                 
 batch_normalization_1 (Bat  (None, 75, 75, 64)       

INFO:tensorflow:Assets written to: modelo_neumonia.final\assets


In [5]:
# Entrenar el modelo
history = model.fit(
    train_generator,
    epochs=7,
    validation_data=val_generator
)

Epoch 1/7



163/163 [==============================] - 170s 988ms/step - loss: 0.4660 - accuracy: 0.8786 - val_loss: 50.4661 - val_accuracy: 0.5000
Epoch 2/7
163/163 [==============================] - 126s 774ms/step - loss: 0.1340 - accuracy: 0.9607 - val_loss: 72.3236 - val_accuracy: 0.5000
Epoch 3/7
163/163 [==============================] - 121s 740ms/step - loss: 0.1053 - accuracy: 0.9680 - val_loss: 16.3757 - val_accuracy: 0.5000
Epoch 4/7
163/163 [==============================] - 123s 753ms/step - loss: 0.0600 - accuracy: 0.9780 - val_loss: 6.0113 - val_accuracy: 0.5000
Epoch 5/7
163/163 [==============================] - 126s 772ms/step - loss: 0.0647 - accuracy: 0.9783 - val_loss: 31.5294 - val_accuracy: 0.5000
Epoch 6/7
163/163 [==============================] - 124s 762ms/step - loss: 0.0468 - accuracy: 0.9858 - val_loss: 0.9511 - val_accuracy: 0.8750
Epoch 7/7
163/163 [==============================] - 127s 780ms/step - loss: 0.0474 - accuracy: 0.9852 - val_loss: 0.2375 - val_accuracy

In [6]:
# Hacer predicciones en datos de prueba
predictions = model.predict(test_generator)

# Evaluar el modelo en datos de prueba
evaluation = model.evaluate(test_generator)

# Por ejemplo, imprimir la precisión en datos de prueba
print("Test Accuracy:", evaluation[1])

20/20 [==============================] - 6s 310ms/step - loss: 2.3778 - accuracy: 0.7115
Test Accuracy: 0.7115384340286255


In [8]:
import gradio as gr

# Cargar el modelo entrenado
model = tf.keras.models.load_model('modelo_neumonia.final')

# Función para realizar predicciones
def predict_image(img):
    # Manejar el caso de una sola imagen (sin lote)
    if len(img.shape) == 2:  
        img = np.expand_dims(img, axis=0)
    elif len(img.shape) == 3 and img.shape[-1] == 1:
        img = np.expand_dims(img, axis=-1)

    # Convertir la imagen a escala de grises
    img = image.array_to_img(img, data_format='channels_last')
    img = img.convert('L')  # Convertir a escala de grises
    img = img.resize((150, 150))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = img / 255.0  # Normalizar la imagen

    # Realizar la predicción
    prediction = model.predict(img)
    # Realizar la predicción
    prediction = model.predict(img)
    print("Valor de predicción:", prediction[0][0])

    # Devolver la predicción (1: neumonía, 0: normal)
    return 'Neumonía' if prediction[0][0] > 0.5 else 'Normal'


    # Devolver la predicción (1: neumonía, 0: normal)
    return 'Neumonía' if prediction[0][0] > 0.5 else 'Normal'

# Crear una interfaz de Gradio
iface = gr.Interface(
    fn=predict_image,
    inputs="image",
    outputs="text",
    live=True
)

# Ejecutar la interfaz
iface.launch()

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


1/1 [==============================] - 0s 45ms/step
Valor de predicción: 0.5009791
1/1 [==============================] - 0s 32ms/step
Valor de predicción: 0.49975148
1/1 [==============================] - 0s 32ms/step
Valor de predicción: 0.49984214
1/1 [==============================] - 0s 31ms/step
Valor de predicción: 0.49984214
1/1 [==============================] - 0s 30ms/step
Valor de predicción: 0.50096875
1/1 [==============================] - 0s 32ms/step
Valor de predicción: 0.5015393
1/1 [==============================] - 0s 39ms/step
Valor de predicción: 0.50094056
1/1 [==============================] - 0s 33ms/step
Valor de predicción: 0.49984214
1/1 [==============================] - 0s 32ms/step
Valor de predicción: 0.49965838
1/1 [==============================] - 0s 31ms/step
Valor de predicción: 0.5000484
1/1 [==============================] - 0s 31ms/step
Valor de predicción: 0.49984214
1/1 [==============================] - 0s 32ms/step
Valor de predicción: 0.5014

In [61]:
epochs = [i for i in range(7)]
fig , ax = plt.subplots(1,2)
train_acc = history.history['accuracy']
train_loss = history.history['loss']
val_acc = history.history['val_accuracy']
val_loss = history.history['val_loss']
fig.set_size_inches(20,10)

ax[0].plot(epochs , train_acc , 'go-' , label = 'Training Accuracy')
ax[0].plot(epochs , val_acc , 'ro-' , label = 'Validation Accuracy')
ax[0].set_title('Training & Validation Accuracy')
ax[0].legend()
ax[0].set_xlabel("Epochs")
ax[0].set_ylabel("Accuracy")

ax[1].plot(epochs , train_loss , 'g-o' , label = 'Training Loss')
ax[1].plot(epochs , val_loss , 'r-o' , label = 'Validation Loss')
ax[1].set_title('Testing Accuracy & Loss')
ax[1].legend()
ax[1].set_xlabel("Epochs")
ax[1].set_ylabel("Training & Validation Loss")
plt.show()
# Código para guardar la imagen
fig.savefig('training_plot.png')

<Figure size 640x480 with 0 Axes>

<Figure size 2000x1000 with 2 Axes>